# ETL Project (Web Scarping)

### Extract

In [6]:
#import library yang dibutuhkan
import requests #untuk akses url
import pandas as pd #untuk membuat DataFrame
from bs4 import BeautifulSoup #untuk scraping HTML
from sqlalchemy import create_engine #untuk akses database

In [7]:
# akses ke HTML
page = requests.get('https://id.wikipedia.org/wiki/Provinsi_di_Indonesia')
soup = BeautifulSoup(page.content, 'html.parser')

#scraping semua element HTML dengan tag 'table' dan class 'wikitable sortable'
table = soup.find('table', {'class':'wikitable sortable'})

In [39]:
#scraping semua element HTML dengan tag 'tr' di dalam tag 'table'
tr = table.find_all('tr')

#list kosong untuk menyimpan data
data = []

for i in range(2, len(tr)):
    tr_i = tr[i]
    th = tr_i.find_all('th') #scraping semua element HTML dengan tag 'th' di dalam tag 'tr'
    td = tr_i.find_all('td') #scraping semua element HTML dengan tag 'td' di dalam tag 'tr'
    cell ={}
    for j in range(len(th)):
        cell['nama_provinsi'] = th[1].text.replace('\n','')
    for k in range(len(td)):
        if k == 2:
            cell['kode_wilayah'] =td[k].text.replace('\n','')
        elif k == 4:
            cell['ibu_kota'] =td[k].text.split('[')[0].replace('\n','')
        elif k == 6:
            cell['hari_jadi'] =td[k].text.split('[')[0].replace('\n','')
        elif k == 8:
            cell['populasi_per2020'] =td[k].text.replace('\n','')
        elif k == 9:
            cell['luas_per2020_km2'] =td[k].text.split('[')[0].replace('\n','')
        elif k == 10:
            cell['kepadatan_penduduk_per2020'] =td[k].text.replace('\n','')
        elif k == 11:
            cell['ipm_per2020'] =td[k].text.replace('\n','')
    data.append(cell)

In [114]:
#buat dataframe dari list data
df = pd.DataFrame(data)
df.head() #menampilkan 5 data teratas

,nama_provinsi,kode_wilayah,ibu_kota,hari_jadi,populasi_per2020,luas_per2020_km2,kepadatan_penduduk_per2020,ipm_per2020
0,Aceh,11,Banda Aceh,Tidak ada,5.274.871,"56.839,09","91,02","71,99"
1,Sumatra Utara,12,Medan,15 April 1948,14.799.361,"72.460,61","202,78","71,77"
2,Sumatra Barat,13,Padang,1 Oktober 1945,5.534.472,"42.137,70","131,73","72,38"
3,Riau,14,Pekanbaru,9 Agustus 1957,6.394.087,"89.935,90","73,48","72,71"
4,Kepulauan Riau,21,Tanjungpinang,24 September 2002,2.064.564,"8.273,87","251,72","75,59"


### Transform

In [115]:
df.info() #untuk menampilkan informasi data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   nama_provinsi               37 non-null     object
 1   kode_wilayah                37 non-null     object
 2   ibu_kota                    37 non-null     object
 3   hari_jadi                   37 non-null     object
 4   populasi_per2020            37 non-null     object
 5   luas_per2020_km2            37 non-null     object
 6   kepadatan_penduduk_per2020  37 non-null     object
 7   ipm_per2020                 37 non-null     object
dtypes: object(8)
memory usage: 2.4+ KB


In [116]:
#transform type data object ke integer
df['populasi_per2020'] = pd.to_numeric(df['populasi_per2020'].str.replace('.',''), errors='coerce', downcast='signed')

C:\Users\bnmo\AppData\Local\Temp\ipykernel_16408\3172423521.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['populasi_per2020'] = pd.to_numeric(df['populasi_per2020'].str.replace('.',''), errors='coerce', downcast='signed')


In [117]:
#transform type data object ke float
df['luas_per2020_km2'] = pd.to_numeric(df['luas_per2020_km2'].str.translate(str.maketrans({'.':'',',':''})),errors='coerce')/100


In [118]:
df['kepadatan_penduduk_per2020'] = pd.to_numeric(df['kepadatan_penduduk_per2020'].str.translate(str.maketrans({'.':'',',':''})),errors='coerce')/100

In [119]:
df['ipm_per2020'] = pd.to_numeric(df['ipm_per2020'].str.replace(',',''),errors='coerce')/100

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   nama_provinsi               37 non-null     object 
 1   kode_wilayah                37 non-null     object 
 2   ibu_kota                    37 non-null     object 
 3   hari_jadi                   37 non-null     object 
 4   populasi_per2020            34 non-null     float64
 5   luas_per2020_km2            37 non-null     float64
 6   kepadatan_penduduk_per2020  34 non-null     float64
 7   ipm_per2020                 34 non-null     float64
dtypes: float64(4), object(4)
memory usage: 2.4+ KB


In [122]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
populasi_per2020,34.0,7.954250e+06,1.129407e+07,701814.00,2.203904e+06,4188645.500,7.949096e+06,48274162.00
luas_per2020_km2,37.0,5.009157e+04,3.877912e+04,661.23,1.967704e+04,45704.160,6.123750e+04,153413.06
kepadatan_penduduk_per2020,34.0,7.412365e+02,2.709257e+03,9.30,5.344750e+01,102.510,2.580500e+02,15906.52
ipm_per2020,34.0,7.108088e+01,3.901881e+00,60.44,6.950500e+01,71.425,7.230750e+01,80.77


### Load

In [125]:
#inisiasi database
user = 'postgres'
password = '0078'
host = 'localhost'
port = 5432
db_name = 'test_db'


In [127]:
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db_name}")
df.to_sql(name='provinsi_db', con=engine, if_exists="replace", index=False)

37